# Events Stream Example

Minimal example showing the events API. Two cells:
1. Setup: Create agent, session, send message
2. View all events

Prerequisites: API server at `http://localhost:9000`

In [ ]:
import requests, time

API = "http://localhost:9000/v1"

# Create agent
agent = requests.post(f"{API}/agents", json={
    "name": "Events Demo",
    "system_prompt": "You are a helpful assistant. Be very brief."
}).json()
agent_id = agent["id"]
print(f"Agent: {agent_id}")

# Create session
session = requests.post(f"{API}/agents/{agent_id}/sessions", json={}).json()
session_id = session["id"]
print(f"Session: {session_id}")

# Send message (triggers agentic loop)
msg = requests.post(f"{API}/agents/{agent_id}/sessions/{session_id}/messages", json={
    "message": {"content": [{"type": "text", "text": "What is 2+2?"}]}
}).json()
print(f"Message sent: {msg['id']}")

# Wait for processing
print("\nWaiting for agent to respond...")
time.sleep(3)

In [ ]:
# View all events
events = requests.get(f"{API}/agents/{agent_id}/sessions/{session_id}/events").json()

print(f"Total events: {len(events['data'])}\n")
print("="*60)

for e in events["data"]:
    event_type = e["event_type"]
    seq = e["sequence"]
    data = e["data"]
    
    print(f"[{seq}] {event_type}")
    
    # Show message content for message events
    if event_type.startswith("message."):
        content = data.get("content", [])
        for part in content:
            if part.get("type") == "text":
                text = part["text"]
                print(f"    text: {text[:80]}{'...' if len(text) > 80 else ''}")
            elif part.get("type") == "tool_call":
                print(f"    tool: {part['name']}({part['arguments']})")
            elif part.get("type") == "tool_result":
                print(f"    result: {part.get('result', part.get('error'))}")
    
    print()

In [ ]:
# Cleanup
requests.delete(f"{API}/agents/{agent_id}/sessions/{session_id}")
requests.delete(f"{API}/agents/{agent_id}")
print("Cleaned up agent and session")